In [132]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [133]:
train_dataset = dsets.CIFAR100(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = dsets.CIFAR100(root='./data', train=False, transform=transforms.ToTensor(), download=False)

In [134]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Own Data Loader

In [135]:
def get_data(images_folder):
    images = []
    for i in os.listdir(images_folder):
        extension = i.split('.')[-1]
        if extension == 'jpg' or extension == 'png':
            img = cv2.imread(os.path.join(images_folder, i), cv2.IMREAD_COLOR)
            img = cv2.resize(img, (200,200))
            images.append(img)
    return np.array(images)

def data_loader(images, batch_size):
    np.random.shuffle(images)
    n_batches = int(len(images)/batch_size)
    return np.array_split(images, n_batches)

my_train_data = get_data(r'C:\Users\trizo\Pictures\WinPics\Backgrounds\1')
my_train_loader = data_loader(my_train_data, 5)
print(len(my_train_loader))
print(my_train_loader[0].shape)

3
(5, 200, 200, 3)


## Iteratable data

In [136]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Defining Model

In [137]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        # Relu
        self.relu = nn.ReLU()
        # Max pool
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(64 * 8 * 8, 100)
    
    def forward(self, x):
        # Convolution 1
        out = self.maxpool(self.relu(self.cnn1(x)))
        # Convolution 2
        out = self.maxpool(self.relu(self.cnn2(out)))
        
        # Resize
        # Original size: (Batch_Size, 32, 8, 8)
        # out.size(0): Batch_Size
        # New out size: (Batch_Size, 32*8*8)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

In [138]:
model = CNNModel().to('cuda')
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [139]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images.to('cuda'))
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels.to('cuda'))
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images
                images = images.requires_grad_()
                
                # Forward pass only to get logits/output
                outputs = model(images.to('cuda'))
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels.to('cuda')).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 3.1687095165252686. Accuracy: 26
Iteration: 1000. Loss: 2.3829879760742188. Accuracy: 32
Iteration: 1500. Loss: 2.5508384704589844. Accuracy: 36
Iteration: 2000. Loss: 2.1158907413482666. Accuracy: 37
Iteration: 2500. Loss: 1.825897216796875. Accuracy: 38
Iteration: 3000. Loss: 1.9829840660095215. Accuracy: 38
